# DIY Skywater 130nm Bandgap Reference Voltage Circuit

Copyright 2022 John William Kustin

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.



By John Kustin


Last Updated: Nov 20 2022

*Assumptions*: Using an Ubuntu machine with sudo access. Have the latest Python. 


**IMPORTANT NOTE** You have to edit `bandgapReferenceCircuit/xschemrc` to replace my paths for skywater130 data and xschem libraries with your paths.

## How to use this Notebook:
1. Install `ngspice`, `xschem`, open_pdks, and the skywater 130nm pdk
2. Clone [the bandgap repository](https://github.com/johnkustin/bandgapReferenceCircuit)
3. Characterize the available transistors with `characterize.py`


    `cd pyMOSChar`


    `python3 characterize.py sky130_fd_pr__nfet_01v8 sky130_fd_pr__pfet_01v8 ngspice --modelFilePath /path-to-skywater-here/skywater-pdk/libraries/sky130_fd_pr/latest/models/sky130.lib.spice tt`


4. Characterize the available bipolar PNP model


    `cd bandgapReferenceCircuit/`


    `xschem &`

    
    Open->`bandgapReferenceCircuit/schematics/bipolar_characterization_temp_and_vbe.sch`


    Netlist and Simulate the schemtaic. Make sure you changed `xschemrc` 
5. Size the remaining bandgap circuit, including the amplifier, current mirrors, etc. (note: gm/Id based design can be used with lookupMos.py)

### Circuit Design

1. Bandgap Design
Read this paper: 
H. Banba et al., "A CMOS bandgap reference circuit with sub-1-V operation," in IEEE Journal of Solid-State Circuits, vol. 34, no. 5, pp. 670-674, May 1999, doi: 10.1109/4.760378.
https://ieeexplore.ieee.org/document/760378

It is important to characterize the bipolar device in the PDK in order to know the temperature coefficient.

2. Remaining Circuit Design
a. The amplifier is the source of loop gain in a feedback control loop. Check outo the small signal form to see negative feedback.
*Thought Provoking Question:* The amplifier is differential so there are two loops, negative and positive, from input to output. Why does one loop need a capacitor on it? How would removing the capacitor affect stability?
b. The amplifier is self-biased. You should work out, with the square law model and extracted model parameters, what is the dc voltage of the amplifier output? This node is the same as the current mirror input, which feeds into another current mirror. What is the relationship between the output dc voltage from an input dc common mode voltage? The range of that function impacts headroom of the amplifier.
c. The circuit requires start-up circuitry for *reliable* startup. Think about how this is a primitive digital control bit. 

My design can be understood through particular attention to tsmc_bandgap_real.sch. You need xschem with the xschem skywater-130 library to really read this file.

The tools I used for design are 
pyMOSchar/characterize.py to characterize NFET and PFET devices,
pyMOSchar/lookupMOS.py to use look-up tables of small signal parameters for many arrayed values of VGS, VDS, VSB, W, L. Another file, pyMOSchar/lookupExamples.py, may also help.

That's it. I hope you have fun! The rest of the repository is related to the bandgap I personally taped out during Stanford's Tapeout Series (https://priyanka-raina.github.io/teaching/)

## Interactive Code

Interact with some NFET and PFET data I generated. 


The location of the raw data is: `bandgapReferenceCircuit/pyMOSChar/sky130.mos.jupyterData.dat`.


Read the code below this cell and run it. This data only has data for pairs of (width, length) = {(1, 0.9), (2, 1), (5, 1.1)} for nmos and (width, length) = {(2, 0.9), (4, 1), (10, 1.1)} for pmos. **These numbers have units of micron**

In [ ]:
from lookupMOS import lum
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pdb

filename = '../pyMOSChar/sky130.mos.jupyterData.dat' 
lk = lum(filename)  
VGS = np.linspace(0, 1.8, endpoint=True)
matplotlib.rc('xtick',labelsize=15)
matplotlib.rc('ytick',labelsize=15)
types = ['nfet', 'pfet']
for idx in range(len(types)):
    typ = types[idx]
    widths = lk.mosDat[typ]['width']
    lengths = lk.mosDat[typ]['length']
    id = np.squeeze(lk.lookup(typ, 'id', l=[l for l in lengths], vds=1.8 ,vsb=0, vgs=VGS))
    gm = np.squeeze(lk.lookup(typ, 'gm', l=[l for l in lengths], vds=1.8,vsb=0, vgs=VGS))
    vt = np.squeeze(lk.lookup(typ, 'vt', l=[l for l in lengths], vds=1.8,vsb=0, vgs=VGS))
    fT = np.squeeze(lk.lookup(typ, 'gm/cgg', l=[l for l in lengths], vds=1.8, vgs=VGS)/2/np.pi)
    K = [id[i]/(VGS-vt[i])**2 * lengths[i] / widths[i] for i in range(len(lengths))] # = 1/2 * mu_n * C_ox
    K_norm = [K[i] / (lengths[i] / widths[i]) for i in range(len(lengths))]
    figK, axK = plt.subplots()
    axK.set_xlabel('Vov (V)')
    axK.set_ylabel('log10 (1/2 * mu_n * C_ox) / (L / W)')
    axK.set_title('{} Normalized square Law K term'.format(typ))
    indc = (VGS-vt) > 0.
    VGSmesh = np.array(np.meshgrid(VGS, lengths))
    for i, k in enumerate(K_norm):
        idxx = np.squeeze(indc[i])
        x_axis = np.squeeze(VGS[idxx]-vt[i][indc[i]])
        y_axis = np.squeeze(np.log10(k[indc[i]]))
        axK.plot(x_axis, y_axis, label=lengths[i])
    axK.legend()
    fig, ax1 = plt.subplots(3)

    ax1[0].set_xlabel('Vov (V)')
    ax1[0].set_ylabel('gm/Id (S/A)')
    ax1[0].set_title('{} Tradeoff: gm/Id vs. fT.'.format(typ))
    lns1 = []
    lns2 = []
    gmonid = [gm[j]/id[j] for j in range(len(lengths))]
    fom = [gmonid[i]*np.squeeze(fT[i])/1e9 for i in range(len(lengths))]
    for i, gmid in enumerate(gmonid):
        lns1.append(ax1[0].plot(np.squeeze(VGS-vt[i]), np.squeeze(gmid), 'o--', label=lengths[i]))
    ax1[0].legend()
    ax2 = ax1[0].twinx()
    ax2.set_ylabel('fT (GHz)')

    for i, f in enumerate(fT):
        lns2.append(ax2.plot(np.squeeze(VGS-vt[i]), np.squeeze(f)/1e9, '+--', label='fT'))

    ax1[0].grid(True)
    fig.tight_layout()
    for i, gi in enumerate(gmonid):
        ax1[1].plot(np.squeeze(gi), np.squeeze(gi)*np.squeeze(fT[i])/1e9, 'o--', label=lengths[i])
    ax1[1].legend()
    for i, gi in enumerate(gmonid):
        ax1[2].plot(np.squeeze(VGS-vt[i]), np.squeeze(gi)*np.squeeze(fT[i])/1e9, 'o--', label=lengths[i])
    ax1[2].legend()

    ax1[1].set_xlabel('gm/Id (S/A)')
    ax1[1].set_ylabel('gm/Id*fT (S/A * GHz)')
    ax1[1].set_title('{} Figure of Merit.'.format(typ))
    ax1[1].grid(True)
    ax1[2].set_xlabel('Vov (V)')
    ax1[2].set_ylabel('gm/Id*fT (S/A * GHz)')
    ax1[2].set_title('{} Figure of Merit.'.format(typ))
    ax1[2].grid(True)
    fig.suptitle(filename + ' W = 1um')
    fig, ax = plt.subplots()
    ax.set_xlabel('Vgs (V)')
    ax.set_ylabel('I (uA)')
    if typ == 'pfet':
        ax.set_xlabel('-Vgs (V)')
        ax.set_ylabel('-I (uA)')
        VGS = -VGS
    else:
        ax.set_xlabel('Vgs (V)')
        ax.set_ylabel('I (uA)')

    ax.set_title('{} Id vs. Vgs. W = 1um'.format(typ))
    for i,idd in enumerate(id) :
        ax.plot(np.squeeze(VGS), idd*1e6, label=f'{lengths[i]} um')
    ax.legend()
plt.show()


Now that you have the tools to intelligently size transistors, you can design the PFET and NFET portion of the larger bandgap circuit.


After reading the Banba paper, you have the resources to understand how and why the proportional to absolute temperature (PTAT) and complementary to absolute temperature (CTAT) temperature coefficients of the PNP and resistor, respectively, cancel out. 

That is what you are designing for. 

The loop gain from the amplifier has to be large enough (your job to figure out how much) for the assumptions in the Banba paper to hold.

Make sure all of your MOSFETs are in saturation!